https://github.com/OpenBMB/MiniCPM-o \
https://huggingface.co/openbmb/MiniCPM-V-2_6

In [1]:
import os

In [2]:
# Set CUDA_VISIBLE_DEVICES to expose only device 0
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Set CUDA_VISIBLE_DEVICES to expose devices 0, 1, 2, and 3
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

In [3]:
import torch
import pandas as pd
from PIL import Image
from transformers import AutoModel, AutoTokenizer

/data/mn27889/miniconda3/envs/path-opendata/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Reading the Pathology Test Data

In [4]:
questions = pd.read_csv("../pathology_test_data/questions.csv")
questions

,image_name,question,original_responses
0,ck_PTGC_2x.jpeg,How can you best describe the low-power patter...,The lymph node shows a mixed follicular and in...
1,ck_PTGC_2x.jpeg,What are three main differential diagnostic co...,"Based on the low-power pattern, the primary co..."
2,ck_PTGC_5x.jpeg,What is the morphologic alteration being depic...,"The image shows a central enlarged, somewhat i..."
3,ck_PTGC_5x.jpeg,What is the expected immunoarchitecture of the...,This image shows an enlarged secondary follicl...
4,ck_serositis_4x.jpg,What is the most common source for the change ...,"The changes here show extensive serositis, wit..."
5,ck_serositis_4x.jpg,What is the specific anatomic region shown in ...,The right half shows the muscularis propria an...
6,ck_steatohepatitis_100x.jpg,If these histologic changes included conspicuo...,There is fatty liver disease and if concurrent...
7,ck_steatohepatitis_100x.jpg,What would the primary histologic feature to s...,Wilson's disease has many non-specific finding...
8,ck_steatohepatitis_200x.jpg,In an overweight adolescent with mildly increa...,"In the background of steatosis, there is incre..."


In [5]:
images_path = os.path.join(os.getcwd(), "../pathology_test_data/images")
images_list = os.listdir(images_path)
images_list

['ck_steatohepatitis_100x.jpg',
 'ck_PTGC_2x.jpeg',
 'ck_serositis_4x.jpg',
 'ck_steatohepatitis_200x.jpg',
 'ck_PTGC_5x.jpeg']

#### Loading the Model Checkpoint and Image Processor

In [6]:
model = AutoModel.from_pretrained('openbmb/MiniCPM-V-2_6', trust_remote_code=True,
                                   attn_implementation='sdpa', torch_dtype=torch.bfloat16) # sdpa or flash_attention_2, no eager

model = model.cuda().eval()

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 25.76it/s]


In [7]:
tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-V-2_6', trust_remote_code=True)

#### Creating the message structure

In [8]:
messages = []
for index, row in questions.iterrows():
    image_path = os.path.join(images_path, row["image_name"])
    question = row["question"]
    image_pil = Image.open(image_path).convert("RGB")
    
    message = [{'role': 'user', 'content': [image_pil, question]}]    
    messages.append(message)

#### Running the batch inference

In [9]:
output = model.chat(
            image=None,
            msgs=messages,
            tokenizer=tokenizer
        )

/data/mn27889/miniconda3/envs/path-opendata/lib/python3.10/site-packages/transformers/models/auto/image_processing_auto.py:644: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [10]:
output

['The low-power pattern in this image of a lymph node shows a predominantly dense and uniform cellular structure with little to no variation. The cells appear as small, tightly packed clusters without significant gaps or spaces between them. This kind of uniformity is typical of normal tissue structures where the cells are closely associated, which can be seen in healthy lymph nodes. However, for a precise diagnosis, one would typically look at higher magnification (high power) sections to identify specific cell types, patterns of cellularity, and any signs of abnormality such as inflammation, infection, or malignancy.',
 "Three main differential diagnostic considerations for a young teenager with solitary lymph node enlargement could include:\n1. **Infectious causes**: Such as bacterial, viral, or fungal infections that can cause localized lymphadenopathy.\n2. **Benign reactive conditions**: Like reactions to vaccinations, inflammatory bowel disease, or other systemic inflammatory con

#### Storing the output Responses

In [12]:
os.makedirs("./responses", exist_ok=True)
questions['minicpm_v26_response'] = output
questions.to_csv("./responses/minicpm_v26_responses.csv", index=False)